In [ ]:
import numpy as np
# from tqdm.notebook import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
import librosa
from pprint import pprint
import scipy.ndimage
from IPython.display import display, Audio
from activation_learner import ActivationLearner
import scipy.signal
import logging
from unmixdb import UnmixDB
from beat_track import beat_stft

logging.basicConfig(level=logging.INFO)
logging.getLogger("activation_learner").setLevel(logging.DEBUG)

In [ ]:
# Load audios

FS = 22050
# ref_paths = ["linear-mix-1.wav", "linear-mix-2.wav"]
# mix_path = "linear-mix.wav"
# ref_paths = ["original.wav"]
# mix_path = "original.wav"
# mix_path = "boucled.wav"
# ref_paths = ["amen.wav", "high.wav"]
# mix_path = "nuttah.wav"
# ref_paths = ["nuttah.wav"]
# mix_path= "nuttah-timestretch.wav"
# refs = [librosa.load(i, sr=FS)[0] for i in ref_paths]
# mix = librosa.load(mix_path, sr=FS)[0]

## generate chirps
# t = np.linspace(0, 2, int(2 * FS))
# refs = [
    # scipy.signal.chirp(t, 200, 2, 5000) + scipy.signal.chirp(t, 500, 2, 8000),
    # scipy.signal.chirp(t, 4000, 2, 800) + scipy.signal.chirp(t, 8000, 2, f1=600),
# ]
# mix = fade(refs[0], refs[1], int(0.5 * FS))
# mix += librosa.load("nuttah.wav", sr=FS)[0][: len(mix)] * 1

# mix += librosa.load("maya.wav", sr=FS)[0][: len(mix)] * 0.3
# mix /= mix.max()

## load unmixdb
unmixdb = UnmixDB("/data2/anasynth_nonbp/schwarz/abc-dj/data/unmixdb-zenodo")
print(unmixdb.timestretches)
print(unmixdb.fxes)
mixes = dict(filter(lambda i: i[1].timestretch == "stretch" and i[1].fx=="none", unmixdb.mixes.items()))
x = list(mixes.values())[5]
pprint(x)
mix = x.audio(sr=FS)
refs = [unmixdb.refsongs[track['name']].audio(sr=FS) for track in x.tracks]

# ## alignment test
# t = np.arange(FS) / FS
# sig = np.concatenate(
#     [
#         np.sin(2 * np.pi * 2000 * t),
#         np.sin(2 * np.pi * 4000 * t),
#         np.sin(2 * np.pi * 6000 * t),
#     ]
# )
# sig = scipy.signal.chirp(t, 800, t[-1], 2000)
# refs = [sig]
# mix = np.concatenate([np.ones(3 * FS) * 1e-50, sig])

display(Audio(mix, rate=FS))
for i, x in enumerate(refs):
    print(i)
    display(Audio(x, rate=FS))

In [ ]:
model = ActivationLearner(
    mix,
    refs,
    fs=FS,
    additional_dim=0,
    win_len = 2**14,
    hop_len = 2**12,
    n_mels = 512,
    col_mag_threshold=1e-8,
)
losses = []
for i in (pbar := tqdm(range(150))):
    loss = model.iterate()
    losses.append(loss)
    pbar.set_description(f"loss={loss:.2e}")
plt.figure(figsize=(8, 2))
plt.plot(losses)
plt.xlabel("iter")
plt.title("distance")
plt.yscale("log")
plt.show()
model.plot()

import soundfile as sf


def estimate_tracks():
    ret = []
    for i in range(len(refs) + 1):
        xi = model.reconstruct(i)
        ret.append(xi)
        print("track", i)
        display(Audio(xi, rate=FS))
        sf.write(f"estimated-{i}.wav", xi / xi.max(), FS)
    return ret


estimates = estimate_tracks()
# sdr, isr, sir, sar = museval.evaluate(
#     np.expand_dims(np.vstack(refs), axis=-1),
#     np.expand_dims(np.vstack(estimates), axis=-1),
# )

In [ ]:
NFFT//HLEN

In [ ]:
H_conv = model.nmf.H*scipy.ndimage.convolve1d(model.nmf.H, scipy.signal.get_window("hann", Nx=NFFT//HLEN), axis=1, mode="constant")
plt.imshow(H_conv, cmap=CMAP, aspect="auto", origin="lower")
plt.show()
plt.imshow(model.nmf.H, cmap=CMAP, aspect="auto", origin="lower")